In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import logging as log
from enum import Enum
from os.path import join, exists, isfile, isdir, abspath, dirname, basename, realpath
from os import makedirs, listdir, pardir, getcwd
from dataclasses import dataclass, field
from typing import Union

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

##### Helpful functions

In [2]:
def print_log(text: str):
    """
    Prints the log
    """
    print(f"[ log ]: {text}")
    # log.info(f"[ log ]: {text}")

def print_error(text: str):
    """
    Prints the error
    """
    print(f"[ error ]: {text}")
    # log.warn(f"[ error ]: {text}")

#### Get data

In [5]:
parent_dir = abspath(join(join(getcwd(), pardir), pardir))
data_dir = join(parent_dir, 'data')
# data_file = join(data_dir, 'DM-HT.txt')
data_file = join(data_dir, 'DM-HT_small.txt')

sample_size = 50

graph = nx.read_edgelist(data_file, nodetype=str, data=(("weight", float),))

print(f"{graph}")

Graph with 27 nodes and 21 edges


In [6]:
from numpy import matrixlib as npmat

In [12]:
matrix = npmat.asmatrix(np.random.randint(2, size=(4, 4)))
matrix

matrix([[1, 0, 0, 1],
        [0, 0, 1, 1],
        [1, 1, 1, 0],
        [1, 0, 0, 1]])